# Visa woes

It seems there's been a problem with your visa, and the authorities have only just noticed, even though you're coming to the end of your holiday. There's no option but to navigate the bowels of the Foreign Ministry, to find the right series of offices to visit.

For some reason known only to themselves, all the offices have a four-letter code that just happens to be an English word. (All the ministry office codes are given in the file [08-rooms.txt](08-rooms.txt).) An office will give you the authorisation to move to a different office if the codes differ by just one letter. For instance, the authorisation from office `rash` will allow you to move to the office `bash` and you can move from `able` to `axle`.

You can move from office `rash` to `jags` by visiting five offices in total:

```
rash
Bash
basS
baGs
Jags
```

where the capital letters indicate the changed letters in each step. There are other ways of getting from `rash` to `jags`, but there is no shorter route.

# Part 1

Including the offices at both ends, what is the smallest number of offices do you have to visit to get from `coax` to `stay`?

You may have found a route to your goal, but what if `stay` wasn't the right office? How many offices could you get in a few steps from a starting point?

For example, there are 11 offices you can reach in one step from `rash`:

`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, `wash`

There are 47 distinct offices you could reach in up to two steps:

`base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, `wish`

The there are 180 distinct offices reachable in up to three steps from `rash`.

The ministry office codes are still given in the file [08-rooms.txt](08-rooms.txt).

# Part 2

How many different offices could you visit in no more than 10 steps from `coax`?

# Worked example solution: part 1
This is the first of two tasks intended to exercise skills developed in M269. In this case, this is about search. I won't go over the general idea of search itself here, as there are loads of tutorials online about it. Instead, I'll talk about the specific way I've implmeneted it in this solution.

(See below for the <a href="#part2">discussion of part 2</a>.)

## Search
The offices/words can be thought of as a graph of nodes and edges. Each word is a node, and there's an edge between two words if those words differ by one letter. The diagrams below show the connections between words that are one step and two steps away from 'rush'.

| Words one step from 'rush' | Words two steps from 'rush' |
| ------------- | ------------- |
| <a href="rush1.dot.png"><img src="rush1.dot.png" alt="Words one step from 'rush'" style="width: 200px;"/></a>     | <a href="rush2.dot.png"><img src="rush2.dot.png" alt="Words two steps from 'rush'" style="width: 200px;"/></a> |

The task is to find a path, a sequence of words, from the starting word to the destination. This is a classic search problem.

The key data structure is the _agenda_, a set of partial routes. We take a partial route from the agenda and _extend_ it with the words/rooms reachable from the end of the partial route, giving a new set of partial routes. Those are added back into the agenda.

Search finishes either when the partial route we're processing is a solution (i.e. goes form the origin to destination room) or the agenda becomes empty.

For instance, say we're going from `ache` to `ashy`. The initial agenda consists of just one partial route, and that partial route contains just `ache`.

```
ache
```

We take that item off the agenda, extend it (with `acne`, `acme`, `acre`, and `achy` as neighbours of `ache`). When we add those four items to the agenda, we get 

```
ache -> acne
ache -> acme
ache -> acre
ache -> achy
```

We then proces the `ache -> acne` partial path, extending it with `acme` and `acre`, giving the agenda:

```
ache -> acme
ache -> acre
ache -> achy
ache -> acne -> acme
ache -> acne -> acre
```

(Note that while `ache` is adjacent to `acne`, we don't want to create a new partial path to `ache` as we've already visited it in this path.)

We then proces the `ache -> acme` partial path, extending it with `acne` and `acre`, giving the agenda:

```
ache -> acre
ache -> achy
ache -> acne -> acme
ache -> acne -> acre
ache -> acme -> acne
ache -> acme -> acre
```

We then do `ache -> acre` and `ache -> achy` to give:

```
ache -> acne -> acme
ache -> acne -> acre
ache -> acme -> acne
ache -> acme -> acre
ache -> acre -> acne
ache -> acre -> acme
ache -> achy -> ashy
```

`ache -> acne -> acme` has only one valid extension, so we get:

```
ache -> acne -> acre
ache -> acme -> acne
ache -> acme -> acre
ache -> acre -> acne
ache -> acre -> acme
ache -> achy -> ashy
ache -> acne -> acme -> acre
```

We process all the other partial paths in turn until we get to the agenda looking like:
```
ache -> achy -> ashy
ache -> acne -> acme -> acre
ache -> acne -> acre -> acme
ache -> acme -> acne -> acre
ache -> acme -> acre -> acne
ache -> acre -> acne -> acme
ache -> acre -> acme -> acne
```
At this point, the first partial path in the agenda is a solution, so we report success and return that path.

With breadth-first search, we add the newly-discovered partial paths to the end of the agenda, meaning we go through the graph one layer at a time. If we add the new partial paths to the front of the agenda, we have depth-first search, where we explore one trail fully before backtracking and trying another.

There are other clever things we can do, such as sorting the agenda by predicted distance to go, which can speed things up.

## Agenda and paths
As each partial path is a sequence of words in order, it makes sense to store it as a Python `list` of words. For breadth-first and depth-first searches, the agenda is a sequence of partial paths. Again, it makes sense to store this a a Python `list` of partial paths.

For A* search, the agenda is sorted by the sum of path length and predicted cost to complete the path. In this case, it makes sense to represent the agenda by a priority queue, also known as a heap. I decided to use the standard Python `heapq` library, rather than the one used in M269. Partial paths are put on the heap with `heappush` and the partial path with the lowest cost is removed with `heappop`. 

## The graph of words
How to represent the graph of words? In the input, the connections between words are implicit, so we need to write a little procedure that returns all the neighbouring words for the word passed in. 

There's also a choice here: to we calculate and store the explicit graph of words, or do we rediscover the neighbours every time we want to process a word? The first approach consumes space to reduce time; the second uses time to reduce space. 

In this case, as there are only 2300 words and 20,000 connections, it's not a lot of space. The search will be examining lots of nodes, so I took a decision to explicity cache all the word neighbour relations first, then just look them up as needed. 

(It turns out this wasn't a good idea in terms of raw perfromance. Generating the graph takes ~130ms, but seems to give just about no change in performance for any search algorithm. Ho hum, but an example of where a [premature optimistion](https://en.wikiquote.org/wiki/Donald_Knuth) turns out to be costly!)

## Important optimisation: closed sets
If you look at the diagrams and traces above, you'll see that the graph of words is very heavily connected, with several different routes from one word to another. But we often don't need to try out all these alternatives. If we're considering a partial route that ends at a particular word _w_, and we've already found another partial route to _w_ that's no longer than this one, there's no need to continue analysing this one. For instance, in the trace above, there's a route `ache -> acne -> acme`, but we've already found the route `ache -> acme`, so we can discard the `ache -> acne -> acme` alternative without worrying about missing possible solutions.

If we use something like breadth-first search, we know that the first time we encounter a new word is the shortest path to that word. That means that all subsequent times we arrive that that word, we know we can discard that partial path. 

We maintain a `closed set` of the words we've already processed. We can use that set in two places. Either we check the partial path when we take it off the agenda, or we use the closed set to reduce the number of partial paths generated at each step. In the implementation below, I do the latter.

## Another optimisation: `list` or `set` of words?
The obvious way to represent the known words is as a list. But, we don't care about the order of items in the collection of words. All we do with them is check for existence of a word, and iterate through the whole collection. In these cases, the Python `set` is a much more efficient data structure than the Pyhton `list`, as it uses something like a `dict` underneath. This means membership checking is much faster while iteration takes about the same amount of time.

Therefore, rather than using `list`s, we'll use `set`s where possible. In fact, as the set of words doesn't change, we can use the `frozenset` type to indicate that the set is immutable.

In [116]:
import string
import heapq

In [117]:
words = frozenset(w.strip() for w in open('08-rooms.txt').readlines())
len(words)

2336

In [191]:
def adjacents(word):
    return frozenset(word[0:i] + l + word[i+1:]
           for i in range(len(word))
           for l in string.ascii_lowercase
           if l != word[i])

In [192]:
%%timeit
neighbours = {w: frozenset(n for n in adjacents(w) if n in words)
             for w in words}

10 loops, best of 3: 130 ms per loop


In [193]:
sum(len(neighbours[w]) for w in neighbours)

20092

In [233]:
def distance(w1, w2):
    return sum(1 for i in range(len(w1))
               if w1[i] != w2[i])

In [195]:
# def extend(chain):
#     return [chain + [s] for s in neighbours[chain[-1]]
#            if s not in chain]

In [196]:
def extend(chain, closed=None):
    if closed:
        nbrs = set(neighbours[chain[-1]]) - closed
    else:
        nbrs = neighbours[chain[-1]]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [197]:
def extend_raw(chain, closed=None):
    if closed:
        nbrs = set(neighbours[chain[-1]]) - closed
    else:
        nbrs = neighbours[chain[-1]]
    return [chain + [s] for s in nbrs
           if s not in chain]

In [198]:
def bfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None        

In [172]:
def bfs_search_raw(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend_raw(current)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None        

In [173]:
def bfs_search_closed(start, goal, debug=False):
    agenda = [[start]]
    closed = set()
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None   

In [181]:
def bfs_search_closed_raw(start, goal, debug=False):
    agenda = [[start]]
    closed = set()
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend_raw(current, closed)
            agenda = agenda[1:] + successors
    if finished:
        return current
    else:
        return None   

In [127]:
def dfs_search(start, goal, debug=False):
    agenda = [[start]]
    finished = False
    while not finished and agenda:
        current = agenda[0]
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            agenda = successors + agenda[1:]
    if finished:
        return current
    else:
        return None        

In [128]:
def astar_search(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend(current)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None        

In [129]:
# Uses direct lookup of successors, rather than using cached neighbours in the dict
def astar_search_raw(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            successors = extend_raw(current) # Difference here
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None        

In [130]:
def astar_search_closed(start, goal, debug=False):
    agenda = [(distance(start, goal), [start])]
    heapq.heapify(agenda)
    closed = set()
    finished = False
    while not finished and agenda:
        _, current = heapq.heappop(agenda)
        if debug:
            print(current)
        if current[-1] == goal:
            finished = True
        else:
            closed.add(current[-1])
            successors = extend(current, closed)
            for s in successors:
                heapq.heappush(agenda, (len(current) + distance(s[-1], goal) - 1, s))
    if finished:
        return current
    else:
        return None   

In [234]:
astar_search('coax', 'stay')

['coax', 'coat', 'boat', 'boar', 'soar', 'star', 'stay']

In [235]:
astar_search_raw('vice', 'wars')

['vice', 'dice', 'dire', 'dare', 'ware', 'wars']

In [201]:
astar_search_raw('boon', 'sell')

['boon', 'boot', 'bolt', 'belt', 'bell', 'sell']

In [236]:
len(astar_search('coax', 'stay'))

7

In [203]:
# len(bfs_search('vice', 'wars'))

In [237]:
len(bfs_search_closed('coax', 'stay'))

7

In [238]:
len(dfs_search('coax', 'stay'))

458

In [239]:
%%timeit
astar_search('coax', 'stay')

1000 loops, best of 3: 605 µs per loop


In [240]:
%%timeit
astar_search_raw('coax', 'stay')

1000 loops, best of 3: 607 µs per loop


In [241]:
%%timeit
astar_search_closed('coax', 'stay')

1000 loops, best of 3: 552 µs per loop


In [243]:
%%timeit
bfs_search('coax', 'stay')

1 loop, best of 3: 4min 25s per loop


In [244]:
%%timeit
bfs_search_raw('coax', 'stay')

1 loop, best of 3: 4min 25s per loop


In [245]:
%%timeit
bfs_search_closed('coax', 'stay')

1 loop, best of 3: 810 ms per loop


In [246]:
%%timeit
bfs_search_closed_raw('coax', 'stay')

1 loop, best of 3: 814 ms per loop


In [247]:
%%timeit
dfs_search('coax', 'stay')

10 loops, best of 3: 26.8 ms per loop


In [248]:
%%timeit
astar_search('amen', 'doff')

1 loop, best of 3: 4.39 s per loop


In [249]:
%%timeit
astar_search_raw('amen', 'doff')

1 loop, best of 3: 4.4 s per loop


In [250]:
%%timeit
astar_search_closed('amen', 'doff')

10 loops, best of 3: 87.4 ms per loop


## Part 2

The example shows that `jags` is reachable in four steps from `rash`. There are 11 words one step away from `rash`: 
`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, and `wash`. 

There are 47 words reachable in one or two steps from `rash`. They are `base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, and `wish`.

There are 180 words reachable in up to three steps from `rash`.

How many words are reachable in up to ten steps from `coax`?


# <a name="part2"></a>Worked example solution: part 2
After all the mucking around with search algorithms, this is actually much easier. It's all to do with building up sets.

The basic idea is to maintain a `set` of reachable words which is extended in passes. In each pass, we find all the neighbours of all the words in the reachable word set, and add those neighbours to the set of reachable words. We just do the number of passes specified in in the task.

In the code below, `reachable` is set of reachable words and `extras` is the set of new words found. As an optimisation, I use the set `boundary` to hold the words added in the previous pass, as the new words to add to `reachable` must be neighbours of a word in the `boundary`: all words that are neighbours of the 'interior' of the set of reachable words have already been added. so there's no need to process them again.

The `trim_extras` flag is for another optimisation: eacy time we add some more words to `extras`, remove words which are already in `reachable`. That will make the later update of `reachable` faster.

This approach is quick, as its based on sets and set membership checking, which is a fast process.

This also suggests another way of solving part 1. Start at depth 1, and find all the words reachable at that depth. If the target word is in the reachable set, report success. If not, increment the depth and find the reachable words again.

This approach does give the right answer of the minimal distance from source to goal words, but it doesn't give any information about the route to take to go from one word to another; the route is something returned by the search algorithms above.

In [217]:
def reachable_in(word, n, trim_extras=False):
    reachable = set()
    boundary = set([word])
    for i in range(n):
        extras = set()
        for w in boundary:
            extras.update(neighbours[w])
        if trim_extras:
            extras.difference_update(reachable)
        reachable.update(boundary)
        boundary = extras.copy()
    return reachable.union(extras).difference(set([word]))

In [218]:
neighbours['rash']

['bash',
 'cash',
 'dash',
 'gash',
 'hash',
 'lash',
 'mash',
 'sash',
 'wash',
 'rush',
 'rasp']

In [219]:
len(reachable_in('rash', 1)), ', '.join(sorted('`{}`'.format(r) for r in reachable_in('rash', 1)))

(11,
 '`bash`, `cash`, `dash`, `gash`, `hash`, `lash`, `mash`, `rasp`, `rush`, `sash`, `wash`')

In [220]:
len(reachable_in('rash', 1)), ', '.join(sorted('<code>{}</code>'.format(r) for r in reachable_in('rash', 1)))

(11,
 '<code>bash</code>, <code>cash</code>, <code>dash</code>, <code>gash</code>, <code>hash</code>, <code>lash</code>, <code>mash</code>, <code>rasp</code>, <code>rush</code>, <code>sash</code>, <code>wash</code>')

In [221]:
len(reachable_in('rash', 2)), ', '.join(sorted('`{}`'.format(r) for r in reachable_in('rash', 2)))

(47,
 '`base`, `bash`, `bask`, `bass`, `bast`, `bath`, `bosh`, `bush`, `case`, `cash`, `cask`, `cast`, `dash`, `dish`, `gash`, `gasp`, `gosh`, `gush`, `hash`, `hasp`, `hath`, `hush`, `lash`, `lass`, `last`, `lath`, `lush`, `mash`, `mask`, `mass`, `mast`, `math`, `mesh`, `mush`, `push`, `ramp`, `rasp`, `ruse`, `rush`, `rusk`, `rust`, `sash`, `sass`, `tush`, `wash`, `wasp`, `wish`')

In [222]:
len(reachable_in('rash', 2)), ', '.join(sorted('<code>{}</code>'.format(r) for r in reachable_in('rash', 2)))

(47,
 '<code>base</code>, <code>bash</code>, <code>bask</code>, <code>bass</code>, <code>bast</code>, <code>bath</code>, <code>bosh</code>, <code>bush</code>, <code>case</code>, <code>cash</code>, <code>cask</code>, <code>cast</code>, <code>dash</code>, <code>dish</code>, <code>gash</code>, <code>gasp</code>, <code>gosh</code>, <code>gush</code>, <code>hash</code>, <code>hasp</code>, <code>hath</code>, <code>hush</code>, <code>lash</code>, <code>lass</code>, <code>last</code>, <code>lath</code>, <code>lush</code>, <code>mash</code>, <code>mask</code>, <code>mass</code>, <code>mast</code>, <code>math</code>, <code>mesh</code>, <code>mush</code>, <code>push</code>, <code>ramp</code>, <code>rasp</code>, <code>ruse</code>, <code>rush</code>, <code>rusk</code>, <code>rust</code>, <code>sash</code>, <code>sass</code>, <code>tush</code>, <code>wash</code>, <code>wasp</code>, <code>wish</code>')

In [223]:
len(reachable_in('rash', 3))

180

In [224]:
len(reachable_in('rash', 10))

2195

In [225]:
len(reachable_in('vice', 10))

2192

In [226]:
%%timeit
len(reachable_in('rash', 10))

100 loops, best of 3: 6.2 ms per loop


In [227]:
%%timeit
len(reachable_in('rash', 10, trim_extras=True))

100 loops, best of 3: 2.92 ms per loop


In [155]:
len(reachable_in('stay', 10))

2188

In [156]:
len(reachable_in('coax', 10))

2193

In [232]:
dist = 0
found = False

while not found and distance < 50:
    dist += 1
    others = reachable_in('coax', distance)
    if 'stay' in others:
        found = True

found, dist

(True, 1)

In [229]:
%%timeit
distance = 0
found = False

while not found and distance < 50:
    distance += 1
    others = reachable_in('coax', distance)
    if 'stay' in others:
        found = True

found, distance

1000 loops, best of 3: 1.26 ms per loop


In [157]:
stay5 = reachable_in('stay', 4)
len(stay5)

280

In [158]:
extras = set()
for w in stay5:
    extras.update(neighbours[w])
extras.difference_update(stay5)
len(extras)

296

In [159]:
' '.join(extras)

'brow chit bees hoot weep cock sots dock boob book boys brag bled eery sows cent crow mead aloe coup wiry less goat well vent deed rood moor belt atop bows draw prep chip sewn whom plus bets sued wets mean news czar newt fees chow reek flue troy pled geek boot akin lock bout leek fell feed shoe club heed bolt root peel thin yock vets crew load wees clue brad been meet pert boos beet legs mews self hock moot tell toad lead shoo jell teed sill skis veil glue went step test crop text peck bran teen crab pier bogs boss leis grow lent quit foam bras prod knot trap cell dram very goad felt chop sack omit poop pelt drag gram peps yens jeep brat prim jets goop tees deep dual bobs beef sits baas dyer lets leaf gees feet chew rend sues gent whet chin whip berm whim yell trig blue peek prop leak lean bent yeps bier drab bell foot heel boon fret moan send tens jock brew crag thaw sick beck gets bend pest loan geed herd skid toot grab pees hair poor rock best bloc pens coat bias fest heft mock lees

In [160]:
astar_search('coax', 'stay')

['coax', 'coat', 'boat', 'boar', 'soar', 'star', 'stay']

In [161]:
%time bfs_search_closed('coax', 'stay')

CPU times: user 848 ms, sys: 0 ns, total: 848 ms
Wall time: 849 ms


['coax', 'coat', 'boat', 'boar', 'soar', 'star', 'stay']

In [162]:
# %time bfs_search('coax', 'stay')

In [163]:
astar_search('czar', 'stay')

['czar', 'tzar', 'tear', 'sear', 'star', 'stay']

In [164]:
# %time bfs_search('czar', 'stay')

In [165]:
# %time bfs_search('coax', 'stay')

In [166]:
rushn = reachable_in('rush', 3)
rushn.add('rush')
len(rushn)

185

In [167]:
links = set()
for w in rushn:
    for n in neighbours[w]:
        if n in rushn:
            links.add(frozenset((n, w)))

with open('rush3.dot', 'w') as f:
    f.write('graph g {\n')
    for l in links:
        lt = tuple(l)
        f.write('"{}" -- "{}";\n'.format(lt[0], lt[1]))
    f.write('}\n')